In [1]:
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from scipy.cluster.vq import whiten, kmeans2

In [3]:
def call_api(uri, result_key):
    json = {}
    try:
        json = requests.get(uri, headers = { 'X-Auth-Token': '861a0bb1df4940b7b945a7d7eb3fdc6c' }).json()
        return json[result_key]
    except:
        print(json)
        return None

In [18]:
teams = call_api('http://api.football-data.org/v1/soccerseasons/426/teams', 'teams')
teams

[{'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/322/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/322/players'},
   'self': {'href': 'http://api.football-data.org/v1/teams/322'}},
  'code': 'HUL',
  'crestUrl': 'http://upload.wikimedia.org/wikipedia/de/a/a9/Hull_City_AFC.svg',
  'name': 'Hull City FC',
  'shortName': 'Hull',
  'squadMarketValue': '122,250,000 €'},
 {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/338/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/338/players'},
   'self': {'href': 'http://api.football-data.org/v1/teams/338'}},
  'code': 'LCFC',
  'crestUrl': 'http://upload.wikimedia.org/wikipedia/en/6/63/Leicester02.png',
  'name': 'Leicester City FC',
  'shortName': 'Foxes',
  'squadMarketValue': '210,500,000 €'},
 {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/340/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/340/

In [26]:
for team in teams:
    value = team['squadMarketValue']
    fixtures = call_api(team['_links']['fixtures']['href'], 'fixtures')
    loss = 0
    for f in fixtures:
        if team['name'] == f['homeTeamName']:
            loss += f['result']['goalsHomeTeam'] if f['result']['goalsHomeTeam'] else 0
        else:
            loss += f['result']['goalsAwayTeam'] if f['result']['goalsAwayTeam'] else 0
    print('Team "{}", {} loss, {} '.format(team['name'], loss, value))
    team['wins'] = wins

Team "Hull City FC", 23 wins, 122,250,000 € 
Team "Leicester City FC", 35 wins, 210,500,000 € 
Team "Southampton FC", 28 wins, 199,000,000 € 
Team "Watford FC", 30 wins, 128,500,000 € 
Team "Middlesbrough FC", 19 wins, 104,750,000 € 
Team "Stoke City FC", 30 wins, 172,250,000 € 
Team "Everton FC", 42 wins, 239,250,000 € 
Team "Tottenham Hotspur FC", 56 wins, 365,500,000 € 
Team "Crystal Palace FC", 33 wins, 157,750,000 € 
Team "West Bromwich Albion FC", 36 wins, 107,600,000 € 
Team "Burnley FC", 28 wins, 66,500,000 € 
Team "Swansea City FC", 32 wins, 106,100,000 € 
Team "Manchester City FC", 74 wins, 518,000,000 € 
Team "Sunderland AFC", 24 wins, 92,500,000 € 
Team "AFC Bournemouth", 36 wins, 121,750,000 € 
Team "Manchester United FC", 38 wins, 534,250,000 € 
Team "Arsenal FC", 73 wins, 468,500,000 € 
Team "Liverpool FC", 55 wins, 387,200,000 € 
Team "Chelsea FC", 55 wins, 514,800,000 € 
Team "West Ham United FC", 35 wins, 241,500,000 € 
